In [1]:
import os
import json
import random

from config import GEMINI_API_KEY, TAVILY_API_KEY

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY


if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
questions_jsonl = "questions_v2.jsonl"

In [4]:
questions_answers_jsonl = "questions_answers_v2.jsonl"

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5 # We want some randomness
)

json_output_parser = JsonOutputParser()

chain = llm | json_output_parser

In [6]:
search = TavilySearchResults(max_results=3)

In [7]:
qa_generation_prompt_template = PromptTemplate(
    input_variables=["QUESTION_STRING", "RETRIEVED_CONTEXT"],
    template="""
You are a university assistant for Singapore University of Technology and Design (SUTD) to answer questions from prospective students about SUTD. Using the question below and a retrieved context, generate a concise and informative answer appropriate for a prospective student to SUTD.

In addition to the answer, output a confidence score from 0 to 1 (as a float) in JSON format. The score should reflect how well the context supports your answer.

Respond in the following JSON format:
{{
  "answer": "<your answer here>",
  "confidence": <float between 0 and 1>
}}

Here is an example:

Question: "What kind of exchange programmes does SUTD offer?"

Context:
--- START OF CONTEXT ---
SUTD offers a range of global exchange programmes including academic exchanges with partner universities in the US, Europe, and Asia. These programmes typically last one term and allow students to take courses overseas while earning credits towards their SUTD degree.
--- END OF CONTEXT ---

Response:
{{
  "answer": "SUTD offers exchange programmes with partner universities in regions like the US, Europe, and Asia. These usually last for one term and allow you to earn credits while studying abroad.",
  "confidence": 0.95
}}

Now, answer the following:

Question: "{QUESTION_STRING}"

Context:
--- START OF CONTEXT ---
{RETRIEVED_CONTEXT}
--- END OF CONTEXT ---

Response:
"""
)


In [8]:
questions = []

with open(questions_jsonl, 'r') as f:
    for line in f:
        try:
            question_dict = json.loads(line)
            questions.append(question_dict['question'])
        except:
            pass

In [9]:
print(len(questions))

313


In [10]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_message

lexicon_question_search_query_pairs = {}

@retry(
    stop=stop_after_attempt(25),
    wait=wait_exponential(multiplier=1, min=1, max=60),
    retry=retry_if_exception_message(match="429|ResourceExhausted")
)
def invoke_chain(chain, prompt):
    return chain.invoke(prompt)

random.shuffle(questions)

for question in questions:
    try:
        query_search_results = search.invoke(question)
    except:
        pass

    prompt = qa_generation_prompt_template.format(
        QUESTION_STRING=question,
        RETRIEVED_CONTEXT=query_search_results 
    )

    results = invoke_chain(chain, prompt)
    
    answer = results["answer"]
    confidence_score = results["confidence"]
    
    try:
        if float(confidence_score) > 0.75:
            with open(questions_answers_jsonl, "a") as f:
                f.write(json.dumps({
                    "question": question,
                    "prompt": prompt,
                    "answer": answer,
                    "context": query_search_results,
                    "confidence": confidence_score
                }) + '\n')
    except:
        pass

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For 